# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# input csv data file
input_data_file = "../output_data/cities.csv"

# Import API key
import sys
sys.path.insert(0, '..')
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv(input_data_file)
weather_df

City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0             albany  42.6001  -73.9662     42.01        48          40   
1             hasaki  35.7333  140.8333     44.60        93          75   
2              tiksi  71.6872  128.8694    -19.08        88          85   
3        petit-bourg  16.1914  -61.5916     78.80        73          75   
4            rikitea -23.1203 -134.9692     80.71        81           1   
..               ...      ...       ...       ...       ...         ...   
543          gedinne  49.9804    4.9367     46.00        93          75   
544     christchurch -43.5333  172.6333     60.01        94         100   
545       farmington  36.7281 -108.2187     41.00        48           1   
546      providencia  28.7167 -111.5833     60.80        44          90   
547  mnogovershinnyy  53.9353  139.9242      9.23        93         100   

     Wind Speed Country        Date  
0         10.36      US  1615301659  
1          4.61      JP  1615301831  
2          7.63      RU  1615301831  
3         16.11      GP  1615301831  
4         13.94      PF  1615301831  
..          ...     ...         ...  
543        6.91      BE  1615301964  
544        2.30      NZ  1615301965  
545       10.36      US  1615301720  
546        5.97      MX  1615301965  
547        3.89      RU  1615301965  

[548 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
fig = gmaps.figure(map_type='ROADMAP')
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[['Lat', 'Lng']]
weights = weather_df['Humidity']
humidity_heat_fig = gmaps.figure(map_type="HYBRID")
heatmap_layer=gmaps.heatmap_layer(locations,weights=weights)

# Add Heatmap layer to map and set max intensity as the mas humidity
humidity_heat_fig.add_layer(heatmap_layer)
heatmap_layer.max_intensity = weather_df['Humidity'].max()
heatmap_layer.point_radius = 8
humidity_heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
valid_weather_df=weather_df.dropna()
vacation_df=valid_weather_df.loc[(valid_weather_df['Lng']<0) &  
                                (valid_weather_df['Max Temp']>55) & 
                                (valid_weather_df['Humidity']>35) &
                                 (valid_weather_df['Humidity']<99) &
                               (valid_weather_df['Wind Speed']<10) &
                               (valid_weather_df['Lat']>35)].copy()
vacation_df

City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
151          teloes  41.3105  -8.1110     60.01        41          39   
169      don benito  38.9563  -5.8616     60.01        54           0   
277  elizabeth city  36.2946 -76.2511     55.99        54           1   
303            vila  42.0304  -8.1588     57.99        43          87   
446         kinston  35.2627 -77.5816     59.00        43           1   

     Wind Speed Country        Date  
151        1.01      PT  1615301868  
169        5.01      ES  1615301872  
277        9.22      US  1615301738  
303        5.01      PT  1615301905  
446        8.05      US  1615301940

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# base params
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

hotel_name=[]
hotel_lat=[]
hotel_lng=[]
hotel_city=[]
count=0

for index,row in vacation_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": str(row['Lat'])+','+str(row['Lng']),
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    try:
        
        print('process hotel info around ' +row['City'])
        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        # convert response to json
        places_data = response.json()

        # pull the hotel name and location from the json file
        hotel_name.append(places_data['results'][0]['name'])
        hotel_lat.append(places_data['results'][0]['geometry']['location']['lat'])
        hotel_lng.append(places_data['results'][0]['geometry']['location']['lng'])
        hotel_city.append(row['City'])
        count+=1
    except:
        print('Hotel not found, Skipping...')

process hotel info around teloes
process hotel info around don benito
process hotel info around elizabeth city
process hotel info around vila
process hotel info around kinston


In [7]:
# get the list of the hotels found and put into the hotel_df DataFrame
print(f'only {count} hotels were found')
hotels_found_df=pd.DataFrame({'City':hotel_city,"Hotel Name":hotel_name,'Hotel Lat':hotel_lat,'Hotel Lng':hotel_lng})
hotel_df=pd.merge(vacation_df,hotels_found_df)
hotel_df

only 5 hotels were found


City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
0          teloes  41.3105  -8.1110     60.01        41          39   
1      don benito  38.9563  -5.8616     60.01        54           0   
2  elizabeth city  36.2946 -76.2511     55.99        54           1   
3            vila  42.0304  -8.1588     57.99        43          87   
4         kinston  35.2627 -77.5816     59.00        43           1   

   Wind Speed Country        Date  \
0        1.01      PT  1615301868   
1        5.01      ES  1615301872   
2        9.22      US  1615301738   
3        5.01      PT  1615301905   
4        8.05      US  1615301940   

                                          Hotel Name  Hotel Lat  Hotel Lng  
0                                     Hotel Monverde  41.321528  -8.120410  
1                     Hotel Vegas Altas | Don Benito  38.955298  -5.878341  
2  Fairfield Inn & Suites by Marriott Elizabeth City  36.298834 -76.251484  
3                                Bungalows da Peneda  42.035508  -8.196077  
4                                         The O'Neil  35.261749 -77.581413

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity_heat_fig.add_layer(marker_layer)
# Display figure
humidity_heat_fig

Figure(layout=FigureLayout(height='420px'))